In [94]:
# libraries for scraping single page
import requests
from bs4 import BeautifulSoup 


In [95]:
#libraries for going through website
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
try:
    from PIL import Image
except ImportError:
    import Image
options = Options()
options.headless = False

In [96]:
import time
import pandas as pd

In [97]:
#TODO
#Get to the page url
#create urls from Ctegories
#for each page use beautiful soup to get gata of image and information and url to that page
#click next page repeat

In [98]:
cwd = os.getcwd()
driver_path=cwd+"/chromedriver" #chromedriver location

<IPython.core.display.Javascript object>

In [99]:
#urls
url_dresses = 'https://www.renttherunway.com/c/'

url_others = 'https://www.renttherunway.com/products/clothing/'
#clothing category
categories = ['top','bottom','dresses','jumpsuit_romper','knit','jacket_coat']
# categories = ['top','bottom']


#Xpaths
close_button='//*[@id="auth-form"]/div/div[1]'
second_close_button="/html/body/div[39]/div/div/button"
total_number_of_pages='//*[@id="page-content"]/div[1]/div[2]/div/div[1]/div[3]/div[3]/div[1]/div[3]/nav/div/p/span[2]'
next_page_link1='//*[@id="page-content"]/div[1]/div[2]/div/div[1]/div[3]/div[3]/div[1]/div[3]/nav/div/p/a'
next_page_link2='//*[@id="page-content"]/div[1]/div[2]/div/div[1]/div[3]/div[3]/div[1]/div[3]/nav/div/p/a[2]'
renting_price='//*[@id="page-content"]/div[1]/div[2]/div/div[1]/div[3]/div[3]/div[2]/div/div[8]/div[2]/div/div/span[2]'
buying_price='//*[@id="page-content"]/div[1]/div[2]/div/div[1]/div[3]/div[3]/div[2]/div/div[9]/div[2]/div/div/span[1]/span'

In [100]:
# gets the RTR main page and send the browser back
def get_main_page(url,browser):
    browser.get(url)
    return browser

# closes the first modal in the main page
def close_modal(browser):
    browser.find_element_by_xpath(close_button).click()
    return browser

# closes the second modal which is a zip code
def close_second_modal(browser):
    browser.find_element_by_xpath(second_close_button).click()
    return browser
    
# gets the total number of pages in each category page
def get_no_pages(browser):
    total_pages=browser.find_element_by_xpath(total_number_of_pages).text
    return int(total_pages.split("/")[-1])

# Goes to the next page two xpaths as th change on the second page
def goto_next_pg(browser,second_page):
    if second_page==False:
        browser.find_element_by_xpath(next_page_link1).click()
    else:
        browser.find_element_by_xpath(next_page_link2).click()
    return browser

"""Function get_single_page_data first creates an array of each information we will scrape for producs ,

(product_url:this takes you to the product detai page,
product_url_which: is the image url of the product that will be used to download the image,
product_name: the name givent to the product ,
product_company: the company that listed the product
price: the min,max rentinf preice and the actual bying price of the product
)

-gets each pages full source code which contains the the product information list whuch is sent to beautiful soup
-beautifulsoup gets the list of all product in a single page and puts information in those arrays 
-the function return a dictionary of arrays that contains products from a single page 
"""
def get_single_page_data(browser,category):
    product_url_arr=[]
    product_img_arr=[]
    product_name_arr=[]
    product_company_arr=[]
    product_rent_price_arr=[]
    product_actual_price_arr=[]
    category_arr=[]
    
    full_page_source= browser.page_source
    soup = BeautifulSoup(full_page_source)
    product_list=soup.findAll("div", {"class": "grid-product-card"})
    
    for items in product_list:
        product_name=items.find("span", {"class": "universal-xsmall grid-product-card-display-name"}).text
        product_comapny=items.find("h2", {"class": "universal-xsmall--semibold grid-product-card-designer"}).text.split(product_name)[0]
        ind_url=items.find("a", {"class": "grid-product-card-detail-link"}).get('href')
        image=items.find("a", {"class": "grid-product-card-detail-link"}).find('img')['src']
        
        product_rent_price=items.find("span", {"rental-price"}).text.split(' to rent')[0]
        product_actual_price=items.find("span", {"universal-xsmall--secondary retail-price"}).text
        
        product_url_arr.append("https://www.renttherunway.com"+ind_url)
        product_img_arr.append(image)
        product_name_arr.append(product_name)
        product_company_arr.append(product_comapny)
        product_rent_price_arr.append(product_rent_price)
        product_actual_price_arr.append(product_actual_price)
        category_arr.append(category)
    context={
    "image_urls":product_img_arr,
    'product_urls':product_url_arr,
    'product_name':product_name_arr,
    'product_company':product_company_arr,
    'buying_price':product_actual_price_arr,
    'renting_price':product_rent_price_arr,
    'category':category_arr
    }
    return context




In [101]:
# # helper fucntion to see test the json
# def write_to_file(data,name):
#     f = open(name, "a")
#     f.write(data)
#     f.close()
#     print("job Done")

In [102]:
# Creates a main dataframe that will contain all the tabular information 
main_df=pd.DataFrame(columns=['image_urls','product_urls','product_name','product_company','buying_price','renting_price','category'])


In [103]:
"""

This is the main workflow( a single browser instance is used for the whole job asn Selenium is not thread safe)
-instantiates the browsert andgoes to the main page ad closes the modal that pops up
-for each category goes to listings for that specific category and closes the second modal that pops up
-gets the toatla number of pages then loops over each page and scrapes product information.
-creates a temporary dataframe for that single page then concatenates to the main daftaframe 
finally sames the  main df in a CSV

"""
try:
    browser = webdriver.Chrome(driver_path,chrome_options=options)
    browser.get("https://www.renttherunway.com")
    try:
        close_modal(browser)
    except:
        print("didnt find modal")
    for category in categories:
        if category == 'dresses':# the url for the dresses category is different form others
            url = url_dresses
        else:
            url = url_others
        browser.get(url+category)
        try:
            close_second_modal(browser)
        except Exception as e:
            print(e)
        num_of_pages=get_no_pages(browser)
        print("total pages",num_of_pages)
        second_page=False
        for page in range(num_of_pages):
            dress_data=get_single_page_data(browser,category)
            temp_df=pd.DataFrame(dress_data)
            main_df=pd.concat([main_df,temp_df])
            print(f'done page {page} out of {num_of_pages} for categoy {category}')
            try:
                goto_next_pg(browser,second_page)
                second_page=True
                time.sleep(10)
            except Exception as e:
                print(e)
                print("in last page")
              
        main_df.to_csv('test.csv')
    browser.quit()
except Exception as e :
    print(e)
    browser.quit()


C:\Users\Tawhid\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: use options instead of chrome_options
  if sys.path[0] == '':


total pages 38
done page 0 out of 38 for categoy top
done page 1 out of 38 for categoy top
done page 2 out of 38 for categoy top
done page 3 out of 38 for categoy top
done page 4 out of 38 for categoy top
done page 5 out of 38 for categoy top
done page 6 out of 38 for categoy top
done page 7 out of 38 for categoy top
done page 8 out of 38 for categoy top
done page 9 out of 38 for categoy top
done page 10 out of 38 for categoy top
done page 11 out of 38 for categoy top
done page 12 out of 38 for categoy top
done page 13 out of 38 for categoy top
done page 14 out of 38 for categoy top
done page 15 out of 38 for categoy top
done page 16 out of 38 for categoy top
done page 17 out of 38 for categoy top
done page 18 out of 38 for categoy top
done page 19 out of 38 for categoy top
done page 20 out of 38 for categoy top
done page 21 out of 38 for categoy top
done page 22 out of 38 for categoy top
done page 23 out of 38 for categoy top
done page 24 out of 38 for categoy top
done page 25 out of 

done page 107 out of 127 for categoy dresses
done page 108 out of 127 for categoy dresses
done page 109 out of 127 for categoy dresses
done page 110 out of 127 for categoy dresses
done page 111 out of 127 for categoy dresses
done page 112 out of 127 for categoy dresses
done page 113 out of 127 for categoy dresses
done page 114 out of 127 for categoy dresses
done page 115 out of 127 for categoy dresses
done page 116 out of 127 for categoy dresses
done page 117 out of 127 for categoy dresses
done page 118 out of 127 for categoy dresses
done page 119 out of 127 for categoy dresses
done page 120 out of 127 for categoy dresses
done page 121 out of 127 for categoy dresses
done page 122 out of 127 for categoy dresses
done page 123 out of 127 for categoy dresses
done page 124 out of 127 for categoy dresses
done page 125 out of 127 for categoy dresses
done page 126 out of 127 for categoy dresses
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="page-cont